In [8]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

In [47]:
url = 'https://www.united.com/en/us/fly/travel/airport/united-club-and-lounge-locations.html'

driver = webdriver.Chrome()
driver.get(url)

wait = WebDriverWait(driver, 10)

dropdown = Select(wait.until(EC.presence_of_element_located((By.ID, "lookup-airport-united-lounges"))))
rows_all = []

for option in dropdown.options[1:]:
    value = option.get_attribute('value')
    print('Selecting:', value)

    dropdown.select_by_value(value)

    time.sleep(2)
    
    tables = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//table")))
    for i, table in enumerate(tables):
        print(f"--- Table {i+1} ---")
        rows = table.find_elements(By.TAG_NAME, "tr")
        for row in rows:
            cols = row.find_elements(By.TAG_NAME, "td")[1:]
            rows_all.append([value, i+1]+[col.text for col in cols])

Selecting: ATL
--- Table 1 ---
Selecting: AUS
--- Table 1 ---
Selecting: bos
--- Table 1 ---
Selecting: ORD
--- Table 1 ---
--- Table 2 ---
Selecting: CLE
--- Table 1 ---
Selecting: DFW
--- Table 1 ---
Selecting: DEN
--- Table 1 ---
--- Table 2 ---
Selecting: FLL
--- Table 1 ---
Selecting: GUM
--- Table 1 ---
Selecting: HKG
--- Table 1 ---
Selecting: HNL
--- Table 1 ---
Selecting: IAH
--- Table 1 ---
--- Table 2 ---
--- Table 3 ---
Selecting: LAS
--- Table 1 ---
Selecting: LAX
--- Table 1 ---
--- Table 2 ---
Selecting: LHR
--- Table 1 ---
Selecting: MEX
--- Table 1 ---
Selecting: MSP
--- Table 1 ---
Selecting: MSY
--- Table 1 ---
Selecting: EWR
--- Table 1 ---
--- Table 2 ---
Selecting: LGA
--- Table 1 ---
Selecting: SNA
--- Table 1 ---
Selecting: MCO
--- Table 1 ---
Selecting: PDX
--- Table 1 ---
Selecting: PHL
--- Table 1 ---
Selecting: PHX
--- Table 1 ---
Selecting: RDU
--- Table 1 ---
Selecting: SAT
--- Table 1 ---
Selecting: SAN
--- Table 1 ---
Selecting: SFO
--- Table 1 ---
--- T

In [ ]:
pd.DataFrame(rows_all)

,0,1,2,3
0,ATL,1,None,None
1,ATL,1,05:00 a.m. – 7:00 p.m. daily,Light snacks\nSelf-service bar\nWi-Fi
2,AUS,1,None,None
3,AUS,1,4:30 a.m. - 8:45 p.m daily,Full-service bar\nLight snacks\nWi-Fi
4,bos,1,None,None
5,bos,1,4:30 a.m. – 8:45 p.m. daily,Full-service bar\nLight snacks\nWi-Fi
6,ORD,1,None,None
7,ORD,1,5:00 a.m. – 10:00 p.m. daily,Full-service bar\nLight snacks\nWi-Fi
8,ORD,1,5:00 a.m. – 9:00 p.m. daily,Full-service bar\nLight snacks\nWi-Fi
9,ORD,1,5:00 a.m. – 9:00 p.m. daily,Full-service bar\nLight snacks\nWi-Fi
